In [41]:
import pandas as pd
import numpy as np
from scipy import stats
import math

from statsmodels.sandbox.stats.multicomp import multipletests 

print(pd.__version__)
print(np.__version__)

1.2.1
1.19.2


# Task 1

Выберите задачи, в которых обязательно нужно применять поправку на множественную проверку гипотез

- все ответы

# Task 2

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:

In [10]:
data = pd.read_csv('AUCs.txt', sep='\t')
data.columns = ['Data', 'C4.5', 'C4.5+m', 'C4.5+cf', 'C4.5+m+cf']
data

,Data,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898
5,iris,0.936,0.931,0.916,0.931
6,liver disorders,0.661,0.668,0.609,0.685
7,lung cancer,0.583,0.583,0.563,0.625
8,lymphography,0.775,0.838,0.866,0.875
9,mushroom,1.000,1.000,1.000,1.000


Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо. 

In [52]:
%%time 
wilcoxon_data = []

for i, lhs_column in enumerate(data.drop(['Data'],axis=1).columns):
    for j, rhs_column in enumerate(data.drop(['Data'],axis=1).columns):
        if i >= j:
            continue
        
        wilcox, p = stats.wilcoxon(data[lhs_column], data[rhs_column])
        wilcoxon_data.append([lhs_column, rhs_column, wilcox, p])

CPU times: user 15.2 ms, sys: 747 µs, total: 15.9 ms
Wall time: 12.7 ms


/home/fantast/anaconda3/envs/coursera/lib/python3.9/site-packages/scipy/stats/morestats.py:2957: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [53]:
model_wilcoxon = pd.DataFrame.from_records(wilcoxon_data)
model_wilcoxon.columns = ['Model_A', 'MOdel_B', 'Wilson', 'p-value']

In [54]:
model_wilcoxon.sort_values(by='p-value', ascending=True)

,Model_A,MOdel_B,Wilson,p-value
0,C4.5,C4.5+m,6.5,0.010757
2,C4.5,C4.5+m+cf,11.0,0.015874
5,C4.5+cf,C4.5+m+cf,10.5,0.025314
3,C4.5+m,C4.5+cf,18.0,0.054329
4,C4.5+m,C4.5+m+cf,22.0,0.327826
1,C4.5,C4.5+cf,43.0,0.861262


# Task 3

Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

- 3

# Task 4

Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?

- Только m

# Task 5

Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [55]:
reject, p_corrected, a1, a2 = multipletests(model_wilcoxon['p-value'], 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [56]:
model_wilcoxon['p_corrected'] = p_corrected
model_wilcoxon['reject'] = reject

In [57]:
model_wilcoxon.sort_values(by='p-value', ascending=True)

,Model_A,MOdel_B,Wilson,p-value,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.064543,False
2,C4.5,C4.5+m+cf,11.0,0.015874,0.079372,False
5,C4.5+cf,C4.5+m+cf,10.5,0.025314,0.101254,False
3,C4.5+m,C4.5+cf,18.0,0.054329,0.162986,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.655651,False
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False


# Task 6

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга? 

In [58]:
reject, p_corrected, a1, a2 = multipletests(model_wilcoxon['p-value'], 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [59]:
model_wilcoxon['p_corrected'] = p_corrected
model_wilcoxon['reject'] = reject

In [60]:
model_wilcoxon.sort_values(by='p-value', ascending=True)

,Model_A,MOdel_B,Wilson,p-value,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.047623,True
2,C4.5,C4.5+m+cf,11.0,0.015874,0.047623,True
5,C4.5+cf,C4.5+m+cf,10.5,0.025314,0.050627,False
3,C4.5+m,C4.5+cf,18.0,0.054329,0.081493,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.393391,False
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False


# Task 7

Насколько корректно, на ваш взгляд, применение метода Бенджамини-Хохберга в этой задаче?

- Всегда не подходит, коректно тоже. Нужно брать не коректно

# Task 8

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга? 

- Брать гиперпараметр и (или) не обращать внимания
- При параметре